using the alternate second innnings features, where second innings history of opponent, location and recent is considered

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score
import pickle

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
second_inn_feature_df = pd.read_csv('csv_data/feature_second_innings_alt.csv',parse_dates=['match_date'],date_parser=custom_date_parser)

In [3]:
second_inn_feature_df.dropna(inplace=True)

In [4]:
first_inn_result_df_train=pd.read_csv('csv_data/result_df_tran.csv',parse_dates=['match_date'],date_parser=custom_date_parser)
first_inn_result_df_test=pd.read_csv('csv_data/result_df_test.csv',parse_dates=['match_date'],date_parser=custom_date_parser)
lr_first_innings=pickle.load(open('first_innings_linear_regression.pkl','rb'))
scaler_first_innings = pickle.load(open('first_innings_linear_regression_scaler.pkl','rb'))

In [5]:
first_innings_feature_columns = ['team_score', 'opponent_score', 'location_base', 'location_mean','batsman_mean', 'batsman_max', 'bowler_mean']

In [6]:
second_inn_feature_train = second_inn_feature_df[second_inn_feature_df['is_train']==True]
second_inn_feature_test = second_inn_feature_df[second_inn_feature_df['is_train']==False]

In [7]:
second_inn_feature_train.columns

Index(['match_id', 'match_date', 'team', 'opponent', 'location', 'team_score',
       'opponent_win_ratio', 'location_win_ratio', 'last_5_match_win_ratio',
       'batsman_mean', 'batsman_max', 'bowler_mean', 'bowler_max', 'is_train',
       'noise', 'target_score', 'runs_scored', 'win'],
      dtype='object')

In [8]:
second_innings_feature_columns = ['team_score',
                                  'opponent_win_ratio', 'location_win_ratio', 'last_5_match_win_ratio', 
                                  'batsman_mean', 'batsman_max', 'bowler_mean',
                                  'bowler_max','target_score']

# second_innings_feature_columns = ['team_score',
#         'batsman_mean', 'batsman_max', 'bowler_mean',
#        'target_score']

second_innings_target = ['win']

In [9]:
second_innings_scaler = StandardScaler()
x_train = second_innings_scaler.fit_transform(second_inn_feature_train[second_innings_feature_columns])
y_train = second_inn_feature_train[second_innings_target]

x_test = second_innings_scaler.transform(second_inn_feature_test[second_innings_feature_columns])
y_test = second_inn_feature_test[second_innings_target]


In [10]:
lgr = LogisticRegression()
lgr.fit(x_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression()

In [11]:
y_train_predict_lgr = lgr.predict(x_train)
y_test_predict_lgr = lgr.predict(x_test)

In [12]:
accuracy_score(y_train,y_train_predict_lgr)

0.841711229946524

In [13]:
accuracy_score(y_test,y_test_predict_lgr)

0.8085106382978723

In [14]:
second_inn_feature_test_copy = pd.DataFrame(second_inn_feature_test)
second_inn_feature_train_copy = pd.DataFrame(second_inn_feature_train)

In [15]:

second_inn_feature_test_copy = second_inn_feature_test_copy.merge(first_inn_result_df_test[['match_id','first_innings_prediction']],
                                   on='match_id',
                                   how='inner')
second_inn_feature_train_copy = second_inn_feature_train_copy.merge(first_inn_result_df_train[['match_id','first_innings_prediction']],
                                   on='match_id',
                                   how='inner')

second_inn_feature_test_copy['target_score'] = second_inn_feature_test_copy['first_innings_prediction']
second_inn_feature_train_copy['target_score'] = second_inn_feature_train_copy['first_innings_prediction']

In [16]:
x_train_copy = second_innings_scaler.transform(second_inn_feature_train_copy[second_innings_feature_columns])
y_train_copy = second_inn_feature_train_copy[second_innings_target]

x_test_copy = second_innings_scaler.transform(second_inn_feature_test_copy[second_innings_feature_columns])
y_test_copy = second_inn_feature_test_copy[second_innings_target]

In [17]:
y_train_copy_predict_lgr = lgr.predict(x_train_copy)
y_test_copy_predict_lgr = lgr.predict(x_test_copy)

In [18]:
accuracy_score(y_train_copy,y_train_copy_predict_lgr)

0.768377253814147

In [19]:
accuracy_score(y_test_copy,y_test_copy_predict_lgr)

0.7339449541284404

# XGBoost

In [20]:
import xgboost as xg
from xgboost import XGBClassifier

In [21]:
xg_model = XGBClassifier(max_depth=3,n_estimators=40)
xg_model.fit(x_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=40, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
y_train_predict_xg = xg_model.predict(x_train)
y_test_predict_xg = xg_model.predict(x_test)

In [23]:
accuracy_score(y_train,y_train_predict_xg),accuracy_score(y_test,y_test_predict_xg)

(0.9176470588235294, 0.7872340425531915)

# Random Forest

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rfc = RandomForestClassifier(max_depth=3)

In [26]:
rfc.fit(x_train,y_train)

<ipython-input-26-b87bb2ba9825>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(x_train,y_train)


RandomForestClassifier(max_depth=3)

In [27]:
y_train_predict_rfc = rfc.predict(x_train)
y_test_predict_rfc = rfc.predict(x_test)

In [28]:
accuracy_score(y_train,y_train_predict_rfc),accuracy_score(y_test,y_test_predict_rfc)

(0.8096256684491978, 0.7730496453900709)

# SVM

In [29]:
from sklearn import svm


In [30]:
clf = svm.SVC(C=0.7,gamma='auto',kernel='linear')
clf.fit(x_train,y_train)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(C=0.7, gamma='auto', kernel='linear')

In [31]:
y_train_predict_svm = clf.predict(x_train)
y_test_predict_svm = clf.predict(x_test)

In [32]:
accuracy_score(y_train,y_train_predict_svm),accuracy_score(y_test,y_test_predict_svm)

(0.8363636363636363, 0.8085106382978723)

# statsmodel

In [33]:
import statsmodels.api as sm 

In [34]:
log_reg = sm.Logit(y_train, sm.add_constant(x_train)).fit() 

Optimization terminated successfully.
         Current function value: 0.377010
         Iterations 7


In [35]:
y_train_predict_stats = np.round(np.array(log_reg.predict(sm.add_constant(x_train))))
y_test_predict_stats = np.round(np.array(log_reg.predict(sm.add_constant(x_test))))

In [36]:
accuracy_score(y_train,y_train_predict_stats),accuracy_score(y_test,y_test_predict_stats)

(0.839572192513369, 0.8156028368794326)

In [37]:
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    win   No. Observations:                  935
Model:                          Logit   Df Residuals:                      925
Method:                           MLE   Df Model:                            9
Date:                Mon, 14 Dec 2020   Pseudo R-squ.:                  0.4559
Time:                        14:30:54   Log-Likelihood:                -352.50
converged:                       True   LL-Null:                       -647.90
Covariance Type:            nonrobust   LLR p-value:                1.984e-121
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2431      0.097      2.512      0.012       0.053       0.433
x1            -0.8703      0.158     -5.502      0.000      -1.180      -0.560
x2             0.0600      0.107      0.562      0.574      -0.149       0.269
x3             0.0935      0.107      0.874      0.382      -0.116       0.303
x4            -0.0091      0.113     -0.081      0.936      -0.230       0.212
x5             3.5745      0.305     11.704      0.000       2.976       4.173
x6            -1.7343      0.209     -8.295      0.000      -2.144      -1.325
x7            -0.4413      0.193     -2.292      0.022      -0.819      -0.064
x8            -0.1189      0.188     -0.631      0.528      -0.488       0.250
x9            -1.6339      0.139    -11.727      0.000      -1.907      -1.361
==============================================================================
"""

In [38]:
#constant,x1,x15,x16,x17,x19